In [1]:
import gradio as gr
import cv2
import numpy as np

c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
weights_path = 'yolo3/yolov3.weights'
config_path = 'yolo3/yolov3.cfg'
names_path = 'yolo3/coco.names'

#yolov3 모델 로드
net = cv2.dnn.readNet(weights_path, config_path)

#클래스 이름 로드
with open(names_path, 'r') as f:
    labels = f.read().strip().split('\n')


In [32]:
def detect_objects(image):
    height, width =image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416,416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]
    detections = net.forward(output_layers)
    box_list =[]
    confidence_list = []
    class_id_list = []
    
    
    for output in detections:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > 0.5:
                box = detection[0:4] * np.array([width, height, width, height])
                (center_x, center_y,  w, h)= box.astype("int")
                x = int(center_x - (w/2))
                y = int(center_y - (h/2))
                
                box_list.append([x, y, int(w), int(h)])
                confidence_list.append(float(confidence))
                class_id_list.append(class_id)
                
    
    index_list= cv2.dnn.NMSBoxes(box_list, confidence_list,0.5, 0.4)
    
    if len(index_list)>0:
        for i in index_list.flatten():
            x,y,w,h = box_list[i]
            label = str(labels[class_id_list[i]])
            confidence = confidence_list[i]
            
            #사각형 그리기
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
            
            #레이블 표시
            cv2.putText(image, f"{label}{confidence:.2f}",(x,y-10),
            cv2.FONT_HERSHEY_COMPLEX,2,(0,255,0),2)    
    
    return image
        
sample_image = cv2.imread('detect-people.png')
detect_objects(sample_image)

#cv2.imshow("image",return_image)

from PIL import Image
pil_image = Image.fromarray(sample_image)
pil_image.show()

def stream_webcam(image):
    return detect_objects(image)


with gr.Blocks() as demo:
    with gr.Row():
        webcam_input = gr.Image(label="실시 간화면",sources="webcam")
        output_image = gr.Image(label="검출 화면")
        
        webcam_input.stream(fn = stream_webcam, inputs=[webcam_input], outputs=[output_image])

        
demo.launch()

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
